In [1]:
import torch
import os
print("PyTorch has version {}".format(torch.__version__))
print(torch.version.cuda)

PyTorch has version 2.2.1+cu121
12.1


In [ ]:
! pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
! pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
! pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu113.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=504020 sha256=a018c6613cacd7ff5d703143bd0bb23c6b1ef90b88f289af4f1945ee4b9eecd4
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu113.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import random
import string
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch_geometric.transforms as T

from plotly import graph_objs as go
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.manifold import TSNE
from sklearn.model_selection import ParameterGrid
from torch_geometric.data import Data, download_url, extract_gz
from torch_geometric.nn import GAE, GCNConv, VGAE
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import train_test_split_edges, negative_sampling, degree


In [ ]:
# set the seeds
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [ ]:
url = 'http://snap.stanford.edu/biodata/datasets/10012/files/DG-AssocMiner_miner-disease-gene.tsv.gz'
extract_gz(download_url(url, '.'), '.')

data_path = "./DG-AssocMiner_miner-disease-gene.tsv"

In [ ]:
# Take a brief look at the structure of the data.
# Note that column names are ["# Disease ID", "Disease Name", "Gene ID"].
df = pd.read_csv(data_path, sep="\t")
print(df.head(), '\n')

In [ ]:
def load_node_mapping(datafile_path, index_col, offset=0):
  df = pd.read_csv(datafile_path, index_col=index_col, sep="\t")
  mapping = {index_id: i + offset for i, index_id in enumerate(df.index.unique())}
  return mapping

def load_edge_list(datafile_path, src_col, src_mapping, dst_col, dst_mapping):
  df = pd.read_csv(datafile_path, sep="\t")
  src_nodes = [src_mapping[index] for index in df[src_col]]
  dst_nodes = [dst_mapping[index] for index in df[dst_col]]
  edge_index = torch.tensor([src_nodes, dst_nodes])
  return edge_index

def initialize_data(datafile_path, num_features=1):
  # Get disease node mapping and gene node mapping.
  # Each node type has its own set of integer ids.
  dz_col, gene_col = "# Disease ID", "Gene ID"
  dz_mapping = load_node_mapping(datafile_path, dz_col, offset=0)
  gene_mapping = load_node_mapping(datafile_path, gene_col, offset=519)

  # Get edge index in terms of the integer indeces assigned to the nodes.
  edge_index = load_edge_list(
      datafile_path, dz_col, dz_mapping, gene_col, gene_mapping)

  # Add the reverse direction (aka make it a undirected graph)
  rev_edge_index = load_edge_list(
      datafile_path, gene_col, gene_mapping, dz_col, dz_mapping)

  # Construct a Data object.
  data = Data()
  data.num_nodes = len(dz_mapping) + len(gene_mapping)
  data.edge_index = torch.cat((edge_index, rev_edge_index), dim=1)
  # pretend we have uniform node features
  data.x = torch.ones((data.num_nodes, num_features))

  return data, gene_mapping, dz_mapping

In [ ]:
# Read data and construct Data object.
data_object, gene_mapping, dz_mapping = initialize_data(data_path)
print(data_object)
print("Number of genes:", len(gene_mapping))
print("Number of diseases:", len(dz_mapping))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.15, is_undirected=True,
                      split_labels=True, add_negative_train_samples=True),
])

train_dataset, val_dataset, test_dataset = transform(data_object)
print("Train Data:\n", train_dataset)
print("Validation Data:\n", val_dataset)
print("Test Data:\n", test_dataset)

In [ ]:
def get_mapping():
  data_path = "./DG-AssocMiner_miner-disease-gene.tsv"
  df = pd.read_csv(data_path, index_col="Disease Name", sep="\t")
  disease_mapping = [index_id for index_id in enumerate(df.index.unique())]
  df = pd.read_csv(data_path, index_col="Gene ID", sep="\t")
  gene_mapping = [index_id[1] for index_id in enumerate(df.index.unique())]
  mapping = disease_mapping + gene_mapping
  return mapping

def visualize_tsne_embeddings(model, data, title, perplexity=30.0,
                              labeled=False, labels=[]):
  """Visualizes node embeddings in 2D space with t-SNE.

  Args: model, pass in the trained or untrained model
        data, Data object, where we assume the first 519 datapoints are disease
          nodes and the rest are gene nodes
        title, title of the plot
        perplexity, t-SNE hyperparameter for perplexity
  """
  model.eval()
  x = data.x
  z = model.encode(x, data.edge_index)
  ax1, ax2 = zip(*TSNE(n_components=2, learning_rate='auto', perplexity=perplexity,
                       init='random').fit_transform(z.detach().cpu().numpy()))

  fig = px.scatter(x=ax1, y=ax2, color=['r']*519 + ['g']*7294,
                   hover_data=[get_mapping()],
                   title=title)

  if labeled:
    for i in labels:
      fig.add_annotation(x=ax1[i], y=ax2[i],
                         text=str(i), showarrow=False)
  fig.show()

def visualize_pca_embeddings(model, data, title, labeled=False, labels=[]):
  """Visualizes node embeddings in 2D space with PCA (components=2)

  Args: model, pass in the trained or untrained model
        data, Data object, where we assume the first 519 datapoints are disease
          nodes and the rest are gene nodes
        title, title of the plot
  """
  model.eval()
  x = data.x
  z = model.encode(x, data.edge_index)

  pca = PCA(n_components=2)
  components = pca.fit_transform(z.detach().cpu().numpy())
  fig = px.scatter(components, x=0, y=1, color=['r']*519 + ['g']*7294,
                   hover_data=[get_mapping()], title=title)

  if labeled:
    for i in labels:
      fig.add_annotation(x=components[:,0][i], y=components[:,1][i],
                         text=str(i), showarrow=False)
  fig.show()

In [ ]:
def plot_roc_curve(model, data):
  """Visualizes ROC curve of model predictions

  Args: model, pass in the trained or untrained model
        data, Data object, where we assume the first 519 datapoints are disease
  """
  model.eval()

  x = data.x
  z = model.encode(x, data.edge_index)

  pos_preds = model.decode(z, data.pos_edge_label_index, sigmoid=True)
  neg_preds = model.decode(z, data.neg_edge_label_index, sigmoid=True)
  preds = torch.cat([pos_preds, neg_preds], dim=0)
  preds = preds.detach().cpu().numpy()

  labels = torch.cat((data.pos_edge_label, data.neg_edge_label), dim=0)
  labels = labels.detach().cpu().numpy()
  fpr, tpr, thresholds = metrics.roc_curve(labels, preds)

  # Using J-statistic: https://en.wikipedia.org/wiki/Youden%27s_J_statistic
  J = tpr - fpr
  ix = np.argmax(J)
  best_thresh = thresholds[ix]
  print('Best Threshold=%f' % (best_thresh))

  roc_auc = metrics.roc_auc_score(labels, preds)

  plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
  plt.plot([0, 1], [0, 1],'r--') # diagonal roc curve of a random classifier
  plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best=%0.2f' % best_thresh)
  plt.xlim(0, 1)
  plt.ylim(0, 1)
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.legend(loc = 'lower right')
  plt.title('ROC curve for model predictions')
  plt.show()

In [ ]:
def plot_training_stats(title, losses, test_auc, test_ap, train_auc, train_ap):
  """Plots evolution of loss and metrics during training

  Args: losses, test_auc, test_ap, train_auc, and train_ap should be lists
    outputted by the training process.
  """
  fig, ax = plt.subplots()
  ax2 = ax.twinx()

  ax.set_xlabel("Training Epochs")
  ax2.set_ylabel("Performance Metric")
  ax.set_ylabel("Loss")

  plt.title(title)
  p1, = ax.plot(losses, "b-", label="training loss")
  p2, = ax2.plot(test_auc, "r-", label="test AUC")
  p3, = ax2.plot(test_ap, "g-", label="test AP")
  p4, = ax2.plot(train_auc, "o-", label="train AUC")
  p5, = ax2.plot(train_ap, "v-", label="train AP")
  plt.legend(handles=[p1, p2, p3, p4, p5])
  plt.show()

In [ ]:
def get_edge_dot_products(data, model, num_dz_nodes=519):
  """
  A pair of nodes (u,v) is predicted to be connected with an edge if the dot
  product between the learned embeddings of u and v is high. This function
  computes and returns the dot product of all pairs of (dz_node, gene_node).

  Args:
    data, the data_object containing the original node featues
    model, the model that will be used to encode the data
    num_dz_nodes, the number of disease nodes; used to differentiate between
      disease and gene node embeddings
  Returns:
    dot_products, a numpy 2D array of shape (num_dz_nodes, num_gene_nodes)
      containing the dot product between each (dz_node, gene_node) pair.
  """
  model.eval()
  x = data.x
  z = model.encode(x, data.edge_index).detach().numpy()
  dz_z = z[:num_dz_nodes, :]
  gene_z = z[num_dz_nodes:, :]

  dot_products = np.einsum('ai,bi->ab', dz_z, gene_z)
  return dot_products   # numpy array of shape (num_dz_nodes, num_gene_nodes)


def get_ranked_edges(data_object, model, num_dz_nodes=519):
  """
  Ranks all potential edges as predicted by the model.

  Args:
    data, the data_object containing the original node featues
    model, the model that will be used to encode the data
    num_dz_nodes, the number of disease nodes; used to differentiate between
      disease and gene node embeddings
  Returns:
    ranked_edge_list, a full edge list ranked by the likelihood of the edge
      being a positive edge, in decreasing order
    ranked_dot_products, a list of the dot products of each edge's node
      embeddings, ranked in decreasing order
  """
  # Get dot products
  edge_dot_products = get_edge_dot_products(data_object, model, num_dz_nodes=num_dz_nodes)
  num_possible_edges = edge_dot_products.shape[0] * edge_dot_products.shape[1]

  # Get indeces, ranked by dot product in descending order. This is a tuple (indeces[0], indeces[1]).
  ranked_edges = np.unravel_index(np.argsort(-1 * edge_dot_products, axis=None), edge_dot_products.shape)
  assert len(ranked_edges[0]) == num_possible_edges

  # Get the corresponding, ranked edge list and ranked dot products. Note that
  # we need to add an offset for the gene_node indeces.
  offset = np.array([np.zeros(num_possible_edges, dtype=int), num_dz_nodes + np.ones(num_possible_edges, dtype=int)]).T
  ranked_edge_list = np.dstack(ranked_edges)[0] + offset
  assert ranked_edge_list.shape[0] == num_possible_edges

  # Get the corresponding ranked dot products
  ranked_dot_products = edge_dot_products[ranked_edges]
  assert ranked_dot_products.shape[0] == num_possible_edges

  return ranked_edge_list, ranked_dot_products


In [ ]:
#@title Dataset Configuration

NUM_FEATURES =   20#@param {type: "integer"}

In [ ]:
data_object.x = torch.ones((data_object.num_nodes, NUM_FEATURES))
print("Using dummy embeddings as initial node features.")
print("Number of features set to ", NUM_FEATURES)

In [ ]:
#@title Model & Training Configuration

HIDDEN_SIZE = 200  #@param {type: "integer"}

OUT_CHANNELS = 20  #@param {type: "integer"}

EPOCHS =   40#@param {type: "integer"}

In [ ]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, hidden_size, out_channels, dropout):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_size, cached=False)
        self.conv_mu = GCNConv(hidden_size, out_channels, cached=False)
        self.conv_logstd = GCNConv(hidden_size, out_channels)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, edge_index):
        x_temp1 = self.conv1(x, edge_index).relu()
        x_temp2 = self.dropout(x_temp1)
        return self.conv_mu(x_temp2, edge_index), self.conv_logstd(x_temp2, edge_index)


vgae_model = VGAE(VariationalGCNEncoder(NUM_FEATURES, HIDDEN_SIZE, OUT_CHANNELS, dropout=0.5))
vgae_model = vgae_model.to(device)

In [ ]:
def vgae_train(train_data, vgae_model, optimizer):
    optimizer.zero_grad()
    vgae_model.train()
    z = vgae_model.encode(train_data.x, train_data.edge_index)
    loss = (
        vgae_model.recon_loss(z, train_data.pos_edge_label_index.to(device)) +
        (1 / train_data.num_nodes) * vgae_model.kl_loss()
    )
    loss.backward(retain_graph=True)
    optimizer.step()
    return float(loss)

@torch.no_grad()
def vgae_test(test_data, vgae_model):
    vgae_model.eval()
    z = vgae_model.encode(test_data.x, test_data.edge_index)
    return vgae_model.test(z, test_data.pos_edge_label_index,
                      test_data.neg_edge_label_index)

In [ ]:
gene_ids_data_path = "./gene_ids.tsv"

gene_ids_df = pd.read_csv(gene_ids_data_path, sep="\t")
gene_ids_df = gene_ids_df.rename(columns={
    "Gene stable ID": "ENSEMBL Gene ID",
    "Gene stable ID version": "ENSEMBL Gene ID Version",
    "NCBI gene (formerly Entrezgene) ID": "Gene ID"})
gene_ids_df = gene_ids_df.loc[:, ["ENSEMBL Gene ID", "Gene ID"]]

# Add new "ENSEMBL Gene ID" column to our existing Gene-Disease Assoc table.
new_df = pd.merge(df, gene_ids_df, left_on="Gene ID", right_on="Gene ID", how="left")
new_df["Gene ID"] = new_df["Gene ID"].astype(int)
print(new_df.loc[:, ["Gene ID", "ENSEMBL Gene ID", ]])  # To confirm mapping.


# Check how many NCBI Gene IDs were unable to be mapped to ENSEMBL Genes IDs.
print("Number of unmapped NCBI Genes:",
      new_df.drop_duplicates(subset="Gene ID")['ENSEMBL Gene ID'].isna().sum())

In [ ]:
# Download the GeneSynopysis dataset directly from the BioSNAP site.
url = 'https://snap.stanford.edu/biodata/datasets/10022/files/G-SynMiner_miner-geneHUGO.tsv.gz'
extract_gz(download_url(url, '.'), '.')

# This table contains info on a select set of genes, with each gene identified
# via their ENSEMBL Gene Id or HGNC Id. We would like to use the "name" as the
# short description of the gene.
data_path = "./G-SynMiner_miner-geneHUGO.tsv.gz"
genes_df = pd.read_csv(data_path, sep="\t")
# print('\n', genes_df.loc[:,["# ensembl_gene_id", "name"]])

# Add the appropriate "name" data to the dataframe for each gene.
full_df = pd.merge(
    new_df, genes_df, left_on="ENSEMBL Gene ID",
    right_on="# ensembl_gene_id", how="left")

# Cut out any extraneous columns from the data frame and rename for easier use.
full_df = full_df.loc[:, ['# Disease ID', 'Disease Name', 'Gene ID', 'name']]
full_df = full_df.rename(columns={
    '# Disease ID': "Disease ID",
    'Disease Name': "Disease Name",
    'Gene ID': "Gene ID",
    'name': "Gene Name",
})

# print(full_df)
# print(full_df.columns)
print("Gene ID <-> Gene Name:\n", full_df.loc[:, ["Gene ID", "Gene Name"]], '\n')

num_nan_ensembl = new_df.drop_duplicates(subset="Gene ID")['ENSEMBL Gene ID'].isna().sum()
num_nan_desc = full_df.drop_duplicates(subset="Gene ID")["Gene Name"].isna().sum()
print("Number of missing ENSEMBL Gene IDs:", num_nan_ensembl)
print("Number of missing gene names:", num_nan_desc)

In [ ]:
def generate_encodings(data_frame, id_col, description_col, node_mapping):
  # Prepare the description data frame.
  desc_df = data_frame.loc[:, [id_col, description_col]]
  desc_df = desc_df.drop_duplicates(subset=id_col)
  desc_df = desc_df[desc_df[description_col].notnull()]

  # Initialize word encoding, and the node encodings.
  num_encoded_words, num_nan, num_nodes = 0, 0, len(node_mapping)
  word_encoding, node_encodings = {}, {}
  excluded_words = [str(i) for i in range(10)]

  # Iterate over the nodes accounted for in node_mapping.
  for node_id, node_index in node_mapping.items():
    # Only attempt to encode if a description is available.
    if node_id not in desc_df[id_col].values:
      num_nan += 1
      continue

    # Get description and break it down to its words.
    description = desc_df.loc[desc_df[id_col] == node_id][description_col].values[0]
    words = [
      word.strip().lower().translate(str.maketrans('', '', string.punctuation))
      for word in description.split()
    ]
    filtered_words = [
      word for word in words if word not in excluded_words
    ]

    # Encode the words.
    encoding = []
    for word in filtered_words:
      if word not in word_encoding:
        word_encoding[word] = num_encoded_words
        num_encoded_words += 1
      encoding.append(word_encoding[word])
    node_encodings[node_index] = encoding

  # Order the encodings as the nodes are ordered in node_mapping.
  final_encoding = [
      node_encodings[i] if i in node_encodings else []
      for i in range(num_nodes)
  ]
  print("Number of nodes without a description:", num_nan)
  return final_encoding, word_encoding


# Encode genes by the one-hot encodings of their descriptions.
# The |gene_encodings| should be in the same order as the genes in |gene_mapping|
print("== GENE NODE ENCODING ==")
gene_encodings, gene_word_encoding = generate_encodings(
    full_df, "Gene ID", "Gene Name", gene_mapping)
print("Number of nodes processed:", len(gene_encodings))
print("Number of words:", len(gene_word_encoding))

# Encode diseases by the one-hot encodings of their descriptions.
# The |dz_encodings| should be in the same order as the genes in |dz_mapping|
print("\n== DISEASE NODE ENCODING ==")
dz_encodings, dz_word_encoding = generate_encodings(
    full_df, "Disease ID", "Disease Name", dz_mapping
)
print("Number of nodes processed:", len(dz_encodings))
print("Number of node words:", len(dz_word_encoding))

In [ ]:
#@title Description Embedding Configuration

GENE_EMB_SIZE =   25#@param {type: "integer"}

DISEASE_EMB_SIZE = 15#@param {type: "integer"}

In [ ]:
class WordToVecModeler(nn.Module):
  # This basic module is roughly based off the tutorial at
  # https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

  def __init__(self, vocab_size, embedding_dim):
    super(WordToVecModeler, self).__init__()
    self.embeddings = nn.Embedding(vocab_size, embedding_dim)
    self.linear1 = nn.Linear(embedding_dim, 128)
    self.linear2 = nn.Linear(128, vocab_size)

  def forward(self, encoded_phrase):
    # |encoded_phrase|: encoded words of the input phrase
    word_embeds = self.embeddings(encoded_phrase)  # List of each word's embedding.
    phrase_embed = torch.sum(word_embeds, 0)
    out = F.relu(self.linear1(phrase_embed))
    out = self.linear2(out)
    log_probs = F.log_softmax(out, dim=0)
    return log_probs


def generate_embeddings(node_encodings, node_word_encoding, embedding_dim):
  num_words = len(node_word_encoding)

  loss_function = nn.NLLLoss()
  model = WordToVecModeler(num_words, embedding_dim)
  optimizer = optim.SGD(model.parameters(), lr=0.001)

  losses = []
  for epoch in range(15):
    total_loss = 0
    for encoded_words in node_encodings:
      if len(encoded_words) > 0:
        model.zero_grad()
        log_probs = model(torch.tensor(encoded_words, dtype=torch.long))

        one_hot_vector = torch.zeros(num_words, dtype=torch.long)
        for code in encoded_words:
          one_hot_vector[code] = 1.0

        loss = loss_function(log_probs, one_hot_vector)

        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    # print(epoch, total_loss)
    losses.append(total_loss)
  print("Losses (total at end of each iteration):\n", losses)  # The loss decreased every iteration over the training data!

  return model.embeddings

In [ ]:
print("Gene Emb Size:", GENE_EMB_SIZE)
gene_embedding = generate_embeddings(gene_encodings, gene_word_encoding, GENE_EMB_SIZE)

print("\nDisease Emb Size:", DISEASE_EMB_SIZE)
dz_embedding = generate_embeddings(dz_encodings, dz_word_encoding, DISEASE_EMB_SIZE)

torch.save(gene_embedding, f="gene_embeds_" + str(GENE_EMB_SIZE))
torch.save(dz_embedding, f="dz_embeds_" + str(DISEASE_EMB_SIZE))


In [ ]:
def compute_node_embeddings(embed_model, encodings):
  # Get statistics of the embeddings.
  weights = embed_model.weight.data.numpy()
  means, stds = np.mean(weights, axis=0), np.std(weights, axis=0)

  node_embeds = []
  for encod in encodings:
    # If encoding is empty, sample a random embedding.
    if len(encod) == 0:
      embed = torch.LongTensor(np.random.normal(loc=means, scale=stds))
    else:
      embed = torch.sum(embed_model(torch.LongTensor(encod)), 0)
    node_embeds.append(embed)

  # Return embeddings of each node.
  embeddings = torch.vstack(node_embeds)
  return embeddings, means, stds


def initialize_featured_dataset(
    gene_encodings, gene_embed_model, dz_encodings, dz_embed_model, edge_index):
  # Get the embeddings for the gene and disease nodes.
  gene_node_embeds, gene_means, gene_stds = compute_node_embeddings(gene_embed_model, gene_encodings)
  dz_node_embeds, dz_means, dz_stds = compute_node_embeddings(dz_embed_model, dz_encodings)

  num_gene_nodes, gene_embed_dim = gene_node_embeds.shape
  num_dz_nodes, dz_embed_dim = dz_node_embeds.shape

  # The node embedding space would be the gene embedding and disease embedding
  # spaces combined. For the gene nodes, the disease embedding dimensions would
  # be randoly sampled, and vice cersa for the disease nodes.
  gene_node_dz_embeds = torch.LongTensor(
      np.random.normal(loc=dz_means, scale=dz_stds, size=(num_gene_nodes, dz_embed_dim)))
  gene_node_embeds = torch.hstack((gene_node_embeds, gene_node_dz_embeds))

  dz_node_gene_embeds = torch.LongTensor(
      np.random.normal(loc=gene_means, scale=gene_stds, size=(num_dz_nodes, gene_embed_dim)))
  dz_node_embeds = torch.hstack((dz_node_gene_embeds, dz_node_embeds))

  # Stack the embeddings to get the full node embeddings matrix, which would be
  # the node features used for our GAE/VGAE models.
  assert gene_node_embeds.shape[1] == dz_node_embeds.shape[1]
  node_embeds = torch.vstack((gene_node_embeds, dz_node_embeds))

  # Construct the homogenous data object.
  data_object = Data(x=node_embeds, edge_index=edge_index)
  return data_object


gene_embed = torch.load("gene_embeds_" + str(GENE_EMB_SIZE))
dz_embed = torch.load("dz_embeds_" + str(DISEASE_EMB_SIZE))

print("Number of gene encodings:", len(gene_encodings))
print("Shape of gene node embeddings:", gene_embed.weight.data.shape)

print("Number of dz encodings:", len(dz_encodings))
print("Shape of disease node embeddings:", dz_embed.weight.data.shape)

data_object_with_features = initialize_featured_dataset(
    gene_encodings, gene_embed, dz_encodings, dz_embed, data_object.edge_index
)
print(data_object_with_features)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.15, is_undirected=True,
                      split_labels=True, add_negative_train_samples=True),
])

train_dataset_with_features, val_dataset_with_features, test_dataset_with_features = transform(data_object_with_features)
print("Train Data w/ Description Embeddings:\n", train_dataset_with_features)
print("Validation Data w/ Description Embeddings:\n", val_dataset_with_features)
print("Test Data w/ Description Embeddings:\n", test_dataset_with_features)

In [ ]:
#@title Model & Training Configuration

MODEL_NAME = "VGAE" #@param ["GAE", "VGAE"] {type:"string"}

HIDDEN_SIZE = 200  #@param {type: "integer"}

OUT_CHANNELS = 20  #@param {type: "integer"}

EPOCHS =   60#@param {type: "integer"}

In [ ]:
if MODEL_NAME == "GAE":
  selected_model = GAE(GCNEncoder(GENE_EMB_SIZE + DISEASE_EMB_SIZE, HIDDEN_SIZE, OUT_CHANNELS, 0.5))
elif MODEL_NAME == "VGAE":
  selected_model = VGAE(VariationalGCNEncoder(GENE_EMB_SIZE + DISEASE_EMB_SIZE, HIDDEN_SIZE, OUT_CHANNELS, 0.5))

print("Model selected:")
selected_model = selected_model.to(device)
print(selected_model)

optimizer = torch.optim.Adam(selected_model.parameters(), lr=0.01)

In [ ]:
losses = []
test_auc = []
test_ap = []
train_aucs = []
train_aps = []

for epoch in range(1, EPOCHS + 1):
  if MODEL_NAME == "GAE":
    loss = gae_train(train_dataset_with_features, selected_model, optimizer)
    auc, ap = gae_test(test_dataset_with_features, selected_model)
    train_auc, train_ap = gae_test(train_dataset_with_features, selected_model)
  else:
    loss = vgae_train(train_dataset_with_features, selected_model, optimizer)
    auc, ap = vgae_test(test_dataset_with_features, selected_model)
    train_auc, train_ap = vgae_test(train_dataset_with_features, selected_model)

  losses.append(loss)
  test_auc.append(auc)
  test_ap.append(ap)
  train_aucs.append(train_auc)
  train_aps.append(train_ap)

  print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}, tAUC: {:.4f}, tAP: {:.4f}, loss:{:.4f}'.format(epoch, auc, ap, train_auc, train_ap, loss))

In [ ]:
plot_training_stats(MODEL_NAME + " with description embeddings", losses, test_auc, test_ap, train_aucs, train_aps)

In [ ]:
plot_roc_curve(selected_model, test_dataset_with_features)

In [ ]:
visualize_tsne_embeddings(
    selected_model, train_dataset_with_features,
    title='Trained Model (using description embeddings): train set embeddings',
    labeled=True, labels=[40, 190, 230, 1830, 260, 110, 280, 1967])

In [ ]:
visualize_pca_embeddings(
    selected_model, train_dataset_with_features,
    title='Trained Model (using description embeddings): train set embeddings',
    labeled=True, labels=[40, 190, 230, 1830, 260, 110, 280, 1967])

In [ ]:
# Select particular model and dataset
data_object_to_analyze = data_object    # Choose from: data_object, data_object_with_features
model_to_analyze = vgae_model            # Choose from: gae_model, vgae_model, selected_model

print("DataObject:\n", data_object_to_analyze)
print("\nModel:\n", model_to_analyze)

In [ ]:
# Get ranked edge list, filtered by the selected node ids.
ranked_edge_list, ranked_dot_products = get_ranked_edges(data_object_to_analyze, model_to_analyze)

# Plot histogram of the dot products
plt.hist(ranked_dot_products, bins=20)
plt.title("Histogram of Dot Products for each Pair of Nodes")
plt.yscale('log')
plt.ylabel("Frequency (log scale)")
plt.xlabel("Dot Product of pairs of nodes")
plt.show()

In [ ]:
# Select for particular examples
select_gene_substrings = ["BRCA"]
select_disease_substrings = ["Alzheimer's Disease","Cancer"]

In [ ]:
# Filter data frame by the selected disease terms, and get their node ids.
if select_disease_substrings:
  dz_regex = "(?i)" + "|".join(select_disease_substrings)
  query_df = full_df[full_df['Disease Name'].str.contains(dz_regex)]
  query_dz_nodes = [dz_mapping[dz_id] for dz_id in query_df['Disease ID'].drop_duplicates().tolist()]
  # print("\nQueried Disease Nodes:\n", query_df['Disease Name'])
else:
  query_dz_nodes = None

# Filter data frame by the selected gene terms, and get their node ids.
if select_gene_substrings:
  gene_regex = "(?i)" + "|".join(select_gene_substrings)
  query_df = full_df[full_df['Gene Name'].notna()]
  query_df = query_df[query_df['Gene Name'].str.contains(gene_regex)]
  query_gene_nodes = [gene_mapping[gene_id] for gene_id in query_df['Gene ID'].drop_duplicates().tolist()]
  # print("\nQueried Gene Nodes:\n", query_df['Gene Name'])
else:
  query_gene_nodes = None


# Get reverse dz and gene mappings, to print out all needed info.
reverse_dz_mapping = {j: i for i,j in dz_mapping.items()}
reverse_gene_mapping = {j: i for i,j in gene_mapping.items()}

print("\nTop Predicted Edges")
top_k = 50
curr_k = 0
for dz_i, gene_i in ranked_edge_list:
  # Skip edges that do not include the selected dz and gene nodes
  if query_dz_nodes and dz_i not in query_dz_nodes:
    continue
  if query_gene_nodes and gene_i not in query_gene_nodes:
    continue

  # Get all the info needed (dz_i and gene_i are the node indeces)
  dz_id, gene_id = reverse_dz_mapping[dz_i], reverse_gene_mapping[gene_i]
  dz_description = full_df[full_df["Disease ID"] == dz_id]["Disease Name"].drop_duplicates().iloc[0]
  gene_description = full_df[full_df["Gene ID"] == gene_id]["Gene Name"].drop_duplicates().iloc[0]
  dot_product = ranked_dot_products[curr_k]

  print('edge=({},{}), \t dotprod={:.2f},\t descriptions=({},{})'.format(dz_i, gene_i, dot_product, dz_description, gene_description))

  curr_k += 1
  if curr_k > top_k:
    break

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Create an empty graph
G = nx.Graph()

# Add nodes for diseases and genes
if query_dz_nodes:
    for dz_node in query_dz_nodes:
        for disease_substring in select_disease_substrings:
            if disease_substring.lower() in full_df[full_df["Disease ID"] == reverse_dz_mapping[dz_node]]["Disease Name"].iloc[0].lower():
                G.add_node(dz_node, type='Disease', label=disease_substring)
                break
if query_gene_nodes:
    G.add_nodes_from(query_gene_nodes, type='Gene')    # Genes are represented as nodes with type 'Gene'

# Add edges
for dz_i, gene_i in ranked_edge_list:
    if query_dz_nodes and dz_i not in query_dz_nodes:
        continue
    if query_gene_nodes and gene_i not in query_gene_nodes:
        continue
    G.add_edge(dz_i, gene_i)  # Edges connect diseases (represented by dz_i) with genes (represented by gene_i)

# Draw the graph with selected nodes
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G)  # Position nodes using Fruchterman-Reingold force-directed algorithm

# Draw disease nodes
for disease_substring in select_disease_substrings:
    disease_nodes = [node for node, attr in G.nodes(data=True) if attr['type'] == 'Disease' and attr['label'] == disease_substring]
    if disease_substring.lower() == "alzheimer's disease":
        node_color = 'lightblue'
    elif disease_substring.lower() == "cancer":
        node_color = 'lightcoral'
    nx.draw_networkx_nodes(G, pos, nodelist=disease_nodes, node_color=node_color, node_size=500, label=disease_substring)

# Draw gene nodes
gene_nodes = [node for node, attr in G.nodes(data=True) if attr['type'] == 'Gene']
nx.draw_networkx_nodes(G, pos, nodelist=gene_nodes, node_color='lightgreen', node_size=500, label='Gene')

# Draw edges
nx.draw_networkx_edges(G, pos)

# Draw labels
nx.draw_networkx_labels(G, pos, font_size=10)

# Set title and legend
plt.title('Predicted Disease-Gene Network (Alzheimer\'s Disease, Cancer, BRCA)')
plt.legend()

# Show plot
plt.show()
